In [68]:
from fenics import *
import meshio
import numpy as np

def detect_boundaries(mesh):
    efun = MeshFunction("size_t",mesh,1)
    AutoSubDomain(lambda x,bc: bc).mark(efun,100)
    edges = np.where(efun.array()==100)[0]

    mesh.init(0,1)
    mesh.init(1,0)
    p2e = mesh.topology()(0,1)
    e2p = mesh.topology()(1,0)

    from collections import deque
    # list of edges to process
    erem = set(edges)
    # components
    components = []
    while erem:
        edge = erem.pop()
        # find component associated to edge
        comp = {edge}
        d = deque([edge])
        while d:
            e0 = d.popleft()
            eoth = {e for v in e2p(e0) for e in p2e(v)
                    if e in erem}
            d.extend(list(eoth))
            comp.update(eoth)
            erem.difference_update(eoth)
            #print(e0,comp,d)
        components.append(list(comp))

    # mark edges with numbers
    markers = []
    for i,comp in enumerate(components):
        marker = (i+1)*100
        efun.array()[comp] = marker
        markers.append(marker)

    return efun,markers

def parametrize_curve(mesh,efun,marker,pt0=None,pt1=None):

    # FIXME add initial point
    mesh.init(0,1)
    mesh.init(1,0)
    p2e = mesh.topology()(0,1)
    e2p = mesh.topology()(1,0)

    edges = set(np.where(efun.array()==marker)[0])

    # NOTE: we cannot assume any order of the edges
    # even by looking at the vertices (in UFC entities
    # are ordered lexicographically).

    e0 = edges.pop()
    p0 = e2p(e0)[0]

    # not using set, we keep the order unchanged
    eord = [e0]
    pord = [p0]
    while edges:
        e0,p0 = eord[-1],pord[-1]
        # edges connected to p0 still in queue
        try:
            enext = next(e for e in p2e(p0) if e in edges)
        except StopIteration:
            break
        edges.remove(enext)
        # the other point
        pnext = next(p for p in e2p(enext) if p!=p0)
        eord.append(enext)
        pord.append(pnext)

    # coordinates of the vertices
    x = mesh.coordinates()
    xv = np.array([x[p,:] for p in pord])
    if pt0 is not None:
        # rotate points and edges to have first point in pt0
        ii = np.argmin(np.linalg.norm(xv-pt0,axis=1))
        pord = np.roll(pord,-ii)
        eord = np.roll(eord,-ii)

    # length of edges
    l = [np.linalg.norm(x[e2p(e)[0],:]-x[e2p(e)[1],:]) for e in eord]
    # coordinate
    s = np.cumsum(l)
    # normalize
    s = np.r_[0.0,s/s[-1]]
    if pt1 is not None:
        # check if pt1 is on the counterclockwise wrt s-coordinate
        # with s < 0.5
        ii = np.argmin(np.linalg.norm(xv-pt1,axis=1))
        if s[ii] < 0.5:
            # flip the order
            s = 1 - s

    # FIXME should we use function defined only on edge?
    P1 = FiniteElement("P",mesh.ufl_cell(),1)
    V = FunctionSpace(mesh,P1)

    sfun = Function(V,name="scoord")
    sfun.vector()[vertex_to_dof_map(V)[pord]] = s[:-1]
    sfun.set_allow_extrapolation(True)

    return sfun

In [77]:
#mm = meshio.read("/Users/pezzuto/OneDrive - USI/archived/crt012_blender_29/crt012-01_LVendo_heart_unif.obj")
mm = meshio.read("RVcut.obj")

msh = Mesh()
editor = MeshEditor()
editor.open(msh,"triangle",2,3)
editor.init_vertices(mm.points.shape[0])
for i,p in enumerate(mm.points):
    editor.add_vertex(i,p)
editor.init_cells(mm.cells_dict['triangle'].shape[0])
for i,c in enumerate(mm.cells_dict['triangle']):
    editor.add_cell(i,c)
editor.close()

c = msh.coordinates().mean(axis=0)
s = np.std(msh.coordinates() - c)
msh.coordinates()[:] = (msh.coordinates() - c)/s

In [78]:
efun,markers = detect_boundaries(msh)
sfun = parametrize_curve(msh,efun,markers[0])

In [79]:
P0 = FiniteElement("DG",msh.ufl_cell(),0)
P1 = FiniteElement("CG",msh.ufl_cell(),1)
V  = FunctionSpace(msh,VectorElement(P1,dim=2))
V0 = FunctionSpace(msh,P0)
Vt = FunctionSpace(msh,TensorElement(P1,shape=(2,2)))
Vt0 = FunctionSpace(msh,TensorElement(P0,shape=(2,2)))

uv = Function(V,name="uv")
Ginv = Function(Vt,name="Ginv")

# initial metric is uniform
Ginv.interpolate(Constant( ((1.0,0.0),(0.0,1.0)) ))

uvtest = TestFunction(V)
F = inner(Ginv*grad(uv),grad(uvtest))*dx

uvbc = Expression(("cos(2*DOLFIN_PI*s)","sin(2*DOLFIN_PI*s)"),
                    s=sfun,element=V.ufl_element())
bcs = [DirichletBC(V,uvbc,efun,markers[0])]
solve(F == 0, uv, bcs)

VV = FunctionSpace(msh,VectorElement(P1))
xfun = Function(VV,name="xyz")
xfun.interpolate(Expression(("x[0]","x[1]","x[2]"),degree=1))

xyz = msh.coordinates().copy()
v2d = vertex_to_dof_map(V)

# estimate metric
msh.coordinates()[:,0:2] = uv.vector()[v2d].reshape((-1,2))
msh.coordinates()[:,2] = 0.0

G = grad(xfun).T * grad(xfun)
G = as_tensor([[G[0,0],G[0,1]],[G[1,0],G[1,1]]])

project(inv(G),function=Ginv)

G00 = project(G[0,0],V0)
G11 = project(G[1,1],V0)
G01 = project(G[0,1],V0)
G10 = project(G[1,0],V0)

msh.coordinates()[:,:] = xyz


      No Jacobian form specified for nonlinear variational problem.
      Differentiating residual form F to obtain Jacobian J = F'.
      Solving nonlinear variational problem.
        Newton iteration 0: r (abs) = 1.202e+02 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
        Newton iteration 1: r (abs) = 4.966e-14 (tol = 1.000e-10) r (rel) = 4.131e-16 (tol = 1.000e-09)
        Newton solver finished in 1 iterations and 1 linear solver iterations.


In [ ]:

solve(F == 0, uv, bcs)

VV = FunctionSpace(msh,VectorElement(P1))
xfun = Function(VV,name="xyz")
xfun.interpolate(Expression(("x[0]","x[1]","x[2]"),degree=1))

xyz = msh.coordinates().copy()
v2d = vertex_to_dof_map(V)

# estimate metric
msh.coordinates()[:,0:2] = uv.vector()[v2d].reshape((-1,2))
msh.coordinates()[:,2] = 0.0

G = grad(xfun).T * grad(xfun)
G = as_tensor([[G[0,0],G[0,1]],[G[1,0],G[1,1]]])

project(inv(G),function=Ginv)

G00 = project(G[0,0],V0)
G11 = project(G[1,1],V0)
G01 = project(G[0,1],V0)
G10 = project(G[1,0],V0)

msh.coordinates()[:,:] = xyz

In [67]:
#msh.coordinates()[:,0:2] = uv.vector()[v2d].reshape((-1,2))
#msh.coordinates()[:,2] = 0.0

with XDMFFile("ffff.xdmf") as f:
    f.parameters['functions_share_mesh'] = True
    f.parameters['rewrite_function_mesh'] = False
    f.write(uv,0.0)

#msh.coordinates()[:,:] = xyz

In [80]:
g00 = G00.vector().get_local()
g11 = G11.vector().get_local()
g01 = G01.vector().get_local()
g10 = G10.vector().get_local()

l1 = Function(V0,name="l1")
l2 = Function(V0,name="l2")
ss = Function(V0,name="s")

for i in range(g00.shape[0]):
    Gmat = np.array([[g00[i],g01[i]],[g10[i],g11[i]]])
    
    (ll1,ll2),_ = np.linalg.eig(Gmat)
    l1.vector()[i] = ll1
    l2.vector()[i] = ll2
    ss.vector()[i] = np.sqrt((ll1+ll2)/2)

with XDMFFile("gggg.xdmf") as f:
    f.parameters['functions_share_mesh'] = True
    f.parameters['rewrite_function_mesh'] = False
    #f.write(l1,0.0)
    #f.write(l2,0.0)
    f.write(ss,0.0)

In [92]:
V = FunctionSpace(msh,P1)
conn = mm.cells_dict['triangle']
nodes = msh.coordinates().copy()
scaling = Function(V,name="metric")
project(ss,function=scaling)

scaling.vector()[vertex_to_dof_map(V)]

np.savez("RV.npz",conn=conn,nodes=nodes,uv=uv.vector()[v2d].reshape((-1,2)),scaling_nodes=scaling.vector()[vertex_to_dof_map(V)])

mo = meshio.Mesh(nodes,{"triangle":conn},point_data={"uv":uv.vector()[v2d].reshape((-1,2))[:,1],"scaling":scaling.vector()[vertex_to_dof_map(V)]})
mo.write("RV.vtk")
